In [1]:
import sys
sys.path.append('./../cbp')
import utils
import cbp
import pandas as pd
import secrets
from cbp import Counties

In [2]:
# texas_api = Counties(state_fips='48', read_from='api', key=secrets.censuskey)
# texas_api.head()
# texas_api.to_csv('texas_cbp_2014.csv', index=False)

In [3]:
texas = Counties(state_fips='48', read_from='csv', filepath='texas_cbp_2014.csv')

In [4]:
texas.head()

,EMP,ESTAB,NAICS2012,NAICS2012_TTL,GEO_TTL,state,county
0,11738,940,00,Total for all sectors,Anderson County,48,001
1,5683,403,00,Total for all sectors,Andrews County,48,003
2,30525,1857,00,Total for all sectors,Angelina County,48,005
3,4131,503,00,Total for all sectors,Aransas County,48,007
4,1539,201,00,Total for all sectors,Archer County,48,009


In [5]:
len(texas)

144585

In [6]:
fips = []
for county in "Bastrop, Caldwell, Hays, Travis, Williamson".split(", "):
    fips.append(texas[(texas.NAICS2012 == '00') & (texas.GEO_TTL.str.contains(county))]['county'].iloc[0])
print fips

['021', '055', '209', '453', '491']


In [7]:
austin_region = texas.two_digit(county=fips)

In [8]:
austin_region.head()

,EMP,ESTAB,NAICS2012,NAICS2012_TTL,GEO_TTL,state,county
10,12475,1175,00,Total for all sectors,Bastrop County,48,021
27,6116,580,00,Total for all sectors,Caldwell County,48,055
104,46675,3660,00,Total for all sectors,Hays County,48,209
225,544038,32217,00,Total for all sectors,Travis County,48,453
244,136393,9252,00,Total for all sectors,Williamson County,48,491


In [17]:
aggs = {
    'EMP':'sum',
    'ESTAB':'sum',
    'NAICS2012_TTL':'first'
}
austin_msa = austin_region.groupby('NAICS2012',as_index=False).agg(aggs)
austin_msa['GEO_TTL'] = 'Austin-Round Rock MSA'

In [18]:
austin_msa.head()

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA
1,11,"Agriculture, Forestry, Fishing and Hunting",28,21,Austin-Round Rock MSA
2,21,"Mining, Quarrying, and Oil and Gas Extraction",216,2440,Austin-Round Rock MSA
3,22,Utilities,116,2282,Austin-Round Rock MSA
4,23,Construction,3665,45334,Austin-Round Rock MSA


In [19]:
texas_total_emp = texas.two_digit().groupby('NAICS2012').agg('sum').EMP
austin_msa_emp = austin_msa.set_index('NAICS2012').EMP
lq = pd.DataFrame(utils.location_quotient(austin_msa_emp, texas_total_emp))
austin_msa.merge(lq, left_on='NAICS2012', right_index=True)

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL,location_quotient
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA,1.000000
1,11,"Agriculture, Forestry, Fishing and Hunting",28,21,Austin-Round Rock MSA,0.069275
2,21,"Mining, Quarrying, and Oil and Gas Extraction",216,2440,Austin-Round Rock MSA,0.160404
3,22,Utilities,116,2282,Austin-Round Rock MSA,0.816960
4,23,Construction,3665,45334,Austin-Round Rock MSA,1.003203
5,31-33,Manufacturing,1299,42592,Austin-Round Rock MSA,0.709544
6,42,Wholesale Trade,2032,34186,Austin-Round Rock MSA,0.886169
7,44-45,Retail Trade,5886,98831,Austin-Round Rock MSA,1.026170
8,48-49,Transportation and Warehousing,790,13551,Austin-Round Rock MSA,0.441452
9,51,Information,1129,31857,Austin-Round Rock MSA,1.831615


In [22]:
texas06 = Counties(state_fips='48', year=2006, read_from='csv', filepath='texas_cbp_2006.csv')

In [23]:
austin_region_06 = texas06.two_digit(county=fips)

In [24]:
aggs = {
    'EMP':'sum',
    'ESTAB':'sum'
}
austin_msa_06 = austin_region_06.groupby('NAICS2012',as_index=False).agg(aggs)
austin_msa_06['GEO_TTL'] = 'Austin-Round Rock MSA'

In [25]:
austin_msa_06

,NAICS2012,ESTAB,EMP,GEO_TTL
0,00,37489,609646,Austin-Round Rock MSA
1,11,26,55,Austin-Round Rock MSA
2,21,168,1439,Austin-Round Rock MSA
3,22,103,789,Austin-Round Rock MSA
4,23,3307,41128,Austin-Round Rock MSA
5,31-33,1202,50637,Austin-Round Rock MSA
6,42,1787,39017,Austin-Round Rock MSA
7,44-45,5067,82717,Austin-Round Rock MSA
8,48-49,622,11020,Austin-Round Rock MSA
9,51,966,24084,Austin-Round Rock MSA


In [36]:
small = austin_msa_06.set_index('NAICS2012').EMP

In [37]:
df = pd.DataFrame(data=small)

In [38]:
df.columns = ['small']

In [39]:
df

,small
NAICS2012,
00,609646
11,55
21,1439
22,789
23,41128
31-33,50637
42,39017
44-45,82717
48-49,11020


In [40]:
larger = texas.two_digit().groupby('NAICS2012').agg('sum').EMP

In [41]:
df['large'] = larger

In [42]:
df

,small,large
NAICS2012,,
00,609646,9598578
11,55,3902
21,1439,195803
22,789,35955
23,41128,581674
31-33,50637,772668
42,39017,496565
44-45,82717,1239705
48-49,11020,395123


In [43]:
total_emp_row = df.loc['00']

In [44]:
total_emp_row.small

609646

In [48]:
all(larger.index == small.index)

True